# 6.3 SMO高效优化算法

## 6.3.2 应用简化版SMO算法处理小规模数据集

In [1]:
import random
import numpy as np
def loadDataSet(fileName):
    dataMat=[]
    labelMat=[]
    with open(fileName) as fr:
        for line in fr.readlines():
            lineArr=line.strip().split('\t')
            dataMat.append([float(lineArr[0]),float(lineArr[1])])
            labelMat.append(float(lineArr[2]))
    return dataMat,labelMat

def selectJrand(i,m):
    j=i
    while(j==i):
        j=int(random.uniform(0,m))
    return j

def clipAlpha(aj,H,L):
    if(aj>H):
        aj=H
    if(L>aj):
        aj=L
    return aj

In [2]:
dataArr,labelArr=loadDataSet('testSet.txt')

In [3]:
def smoSimple(dataMatIn,classLabels,C,toler,maxIter):##toler:ε
    dataMatrix=np.mat(dataMatIn)
    labelMat=np.mat(classLabels).transpose()
    b=0
    m,n=dataMatrix.shape
    alphas=np.mat(np.zeros((m,1)))
    iter=0
    while(iter<maxIter):
        alphaPairsChanged=0
        for i in range(m):
            fxi=float(np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[i,:].T))+b
            Ei=fxi-labelMat[i]
            if((labelMat[i]*Ei<-toler) and (alphas[i]<C)) or ((labelMat[i]*Ei>toler) and (alphas[i]>0)):
                j=selectJrand(i,m)
                fxj=np.multiply(alphas,labelMat).T*(dataMatrix*dataMatrix[j,:].T)+b
                Ej=fxj-labelMat[j]
                alphaIold=alphas[i].copy()
                alphaJold=alphas[j].copy()
                if(labelMat[i]!=labelMat[j]):
                    L=max(0,alphas[j]-alphas[i])
                    H=min(C,C+alphas[j]-alphas[i])
                else:
                    L=max(0,alphas[j]+alphas[i]-C)
                    H=min(C,alphas[j]+alphas[i])                    
                if(L==H):
                    print "L=H"
                    continue
                eta=dataMatrix[i,:]*dataMatrix[i,:].T+\
                    dataMatrix[j,:]*dataMatrix[j,:].T-\
                    2.0*dataMatrix[i,:]*dataMatrix[j,:].T
                if(eta<=0):
                    print "eta<=0"
                    continue
                alphas[j]+=labelMat[j]*(Ei-Ej)/eta
                alphas[j]=clipAlpha(alphas[j],H,L)
                if(abs(alphas[j]-alphaJold)<0.00001):
                    print 'j not moving enough'
                    continue
                alphas[i]+=labelMat[j]*labelMat[i]*(alphaJold-alphas[j])
                b1=-Ei-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[i,:].T-\
                        labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[i,:]*dataMatrix[j,:].T+b
                b2=-Ej-labelMat[i]*(alphas[i]-alphaIold)*dataMatrix[i,:]*dataMatrix[j,:].T-\
                        labelMat[j]*(alphas[j]-alphaJold)*dataMatrix[j,:]*dataMatrix[j,:].T+b
                if(0<alphas[i]<C): 
                    b=b1
                elif(0<alphas[j]<C):
                    b=b2
                else: 
                    b=(b1+b2)/2.0
                alphaPairsChanged+=1
                print "iter: %d i:%d, pairs changed %d"%(iter,i,alphaPairsChanged)
        if(alphaPairsChanged==0): 
            iter+=1
        else: 
            iter=0
        print "iteration number: %d" %iter
    return b,alphas

In [4]:
b,alpha=smoSimple(dataArr,labelArr,0.6,0.001,40)

iter: 0 i:0, pairs changed 1
j not moving enough
L=H
j not moving enough
j not moving enough
L=H
L=H
L=H
j not moving enough
L=H
L=H
j not moving enough
L=H
j not moving enough
j not moving enough
L=H
j not moving enough
L=H
L=H
j not moving enough
iteration number: 0
L=H
L=H
j not moving enough
L=H
L=H
j not moving enough
j not moving enough
L=H
L=H
L=H
L=H
L=H
j not moving enough
iter: 0 i:70, pairs changed 1
j not moving enough
j not moving enough
L=H
iteration number: 0
iter: 0 i:0, pairs changed 1
L=H
L=H
j not moving enough
j not moving enough
L=H
L=H
L=H
j not moving enough
iter: 0 i:70, pairs changed 2
j not moving enough
L=H
L=H
L=H
iteration number: 0
iter: 0 i:0, pairs changed 1
j not moving enough
j not moving enough
L=H
j not moving enough
L=H
j not moving enough
L=H
iter: 0 i:30, pairs changed 2
L=H
j not moving enough
iteration number: 0
j not moving enough
L=H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not movin

In [5]:
b

matrix([[-3.87717646]])

# 6.4 利用完整Platt SMO算法加速优化

In [6]:
class optStruct(object):
    def __init__(self,dataMatIn, classLabels, C, toler, kTup): 
        self.X=dataMatIn
        self.labelMat=classLabels
        self.C=C
        self.toler=toler
        self.m=dataMatIn.shape[0]
        self.alphas=np.mat(np.zeros((self.m,1)))
        self.b=0
        self.eCache=np.mat(np.zeros((self.m,2)))##第一列是是否有效的标识符
        self.K = np.mat(np.zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernelTrans(self.X, self.X[i,:], kTup)
        
def calcEk(oS, k):
    fXk = np.multiply(oS.alphas,oS.labelMat).T*oS.K[:,k] + oS.b
    Ek = fXk - float(oS.labelMat[k])
    return Ek
    
def selectJ(i,oS,Ei):
    maxK=-1
    maxDeltaE=0
    Ej=0
    oS.eCache[i]=[1,Ei]
    validEcacheList=list(np.nonzero(oS.eCache[:,0])[0])
    if(len(validEcacheList)>1):
        for k in validEcacheList:
            if k==i:
                continue
            Ek=calcEk(oS,k)
            deltaE=np.abs(Ei-Ek)
            if(deltaE>maxDeltaE):
                maxK=k
                maxDeltaE=deltaE
                Ej=Ek
        return maxK,Ej
    else:
        j=selectJrand(i,oS.m)
        Ej=calcEk(oS,j)
    return j,Ej

def updateEk(oS,k):
    Ek=calcEk(oS,k)
    oS.eCache[k]=[1,Ek]
    
def kernelTrans(X, A, kTup):
    m,n = X.shape
    K = np.mat(np.zeros((m,1)))
    if(kTup[0]=='lin'): 
        K=X*A.T
    elif(kTup[0]=='rbf'):
        for j in range(m):
            deltaRow = X[j,:] - A
            K[j] = deltaRow*deltaRow.T
        K = np.exp(K/(-1*kTup[1]**2))
    else: 
        raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K
    
def innerLoop(i, oS):
    Ei = calcEk(oS, i)
    if ((oS.labelMat[i]*Ei < -oS.toler) and (oS.alphas[i] < oS.C)) or ((oS.labelMat[i]*Ei > oS.toler) and (oS.alphas[i] > 0)):
        j,Ej=selectJ(i,oS,Ei)
        alphaIold=oS.alphas[i].copy()
        alphaJold=oS.alphas[j].copy()
        if(oS.labelMat[i]!=oS.labelMat[j]):
            L=max(0,oS.alphas[j]-oS.alphas[i])
            H=min(oS.C,oS.C+oS.alphas[j]-oS.alphas[i])
        else:
            L=max(0,oS.alphas[j]+oS.alphas[i]-oS.C)
            H=min(oS.C,oS.alphas[j]+oS.alphas[i])
        if(L==H):
            print "L=H"
            return 0
        eta=oS.X[i,:]*oS.X[i,:].T+\
            oS.X[j,:]*oS.X[j,:].T-\
            2.0*oS.X[i,:]*oS.X[j,:].T
        if(eta<=0):
            print "eta<=0"
            return 0
        oS.alphas[j]+=oS.labelMat[j]*(Ei-Ej)/eta
        oS.alphas[j]=clipAlpha(oS.alphas[j],H,L)
        updateEk(oS,j)
        if(abs(oS.alphas[j]-alphaJold)<0.00001):
            print 'j not moving enough'
            return 0
        oS.alphas[i]+=oS.labelMat[j]*oS.labelMat[i]*(alphaJold-oS.alphas[j])
        updateEk(oS,i)
        b1=-Ei-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,i]-\
                oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[i,j]+oS.b
        b2=-Ej-oS.labelMat[i]*(oS.alphas[i]-alphaIold)*oS.K[i,j]-\
                    oS.labelMat[j]*(oS.alphas[j]-alphaJold)*oS.K[j,j]+oS.b
        if(0<oS.alphas[i]<oS.C): 
            oS.b=b1
        elif(0<oS.alphas[j]<oS.C):
            oS.b=b2
        else: 
            b=(b1+b2)/2.0
        return 1
    else:
        return 0
        
def smoP(dataMatIn, classLabels, C, toler, maxIter,kTup=('lin', 0)):
    oS = optStruct(np.mat(dataMatIn),np.mat(classLabels).transpose(),C,toler, kTup)
    iter = 0
    entireSet = True
    alphaPairsChanged = 0
    while (iter < maxIter):
        alphaPairsChanged = 0
        if entireSet: 
            for i in range(oS.m):        
                alphaPairsChanged += innerLoop(i,oS)
                print "fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            if(alphaPairsChanged==0):
                break
            else:
                entireSet=False
        else:
            nonBoundIs = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerLoop(i,oS)
                print "non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alphaPairsChanged)
            if (alphaPairsChanged == 0): 
                entireSet = True  
        iter += 1
    print "iteration number: %d" % iter
    return oS.b,oS.alphas

In [7]:
dataArr,labelArr=loadDataSet('testSet.txt')
b,alphas=smoP(dataArr,labelArr,0.6,0.001,40)

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 2
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 4
fullSet, iter: 0 i:7, pairs changed 4
fullSet, iter: 0 i:8, pairs changed 5
fullSet, iter: 0 i:9, pairs changed 5
j not moving enough
fullSet, iter: 0 i:10, pairs changed 5
fullSet, iter: 0 i:11, pairs changed 5
fullSet, iter: 0 i:12, pairs changed 5
fullSet, iter: 0 i:13, pairs changed 5
fullSet, iter: 0 i:14, pairs changed 5
fullSet, iter: 0 i:15, pairs changed 5
fullSet, iter: 0 i:16, pairs changed 5
j not moving enough
fullSet, iter: 0 i:17, pairs changed 5
fullSet, iter: 0 i:18, pairs changed 6
fullSet, iter: 0 i:19, pairs changed 6
fullSet, iter: 0 i:20, pairs changed 6
fullSet, iter: 0 i:21, pairs changed 6
fullSet, iter: 0 i:22, pairs changed 6
fullSet, iter: 0 i:23, pairs changed 7
fullSet, iter: 0 i:24, pairs chang

In [8]:
b

matrix([[-2.88322544]])

In [9]:
def calcWs(alphas,dataArr,classLabels):
    X=np.mat(dataArr)
    labelArr=np.mat(classLabels).transpose()
    n,m=X.shape
    w=np.zeros((m,1))
    for i in range(n):
        w+=np.multiply(alphas[i]*labelArr[i],X[i,:].T)
    return w

In [10]:
ws=calcWs(alphas,dataArr,labelArr)

In [11]:
ws

array([[ 0.65899234],
       [-0.29560689]])

## 6.5.3 在测试中使用核函数

In [12]:
def testRbf(k1=1.3):
    dataArr,labelArr=loadDataSet('testSetRBF.txt')
    b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,('rbf',k1))
    dataMat=np.mat(dataArr)
    labelMat=np.mat(labelArr).transpose()
    svInd=np.nonzero(alphas.A>0)[0]
    sVs=dataMat[svInd]
    labelSV=labelMat[svInd]
    print 'there are %d Support Vectors'%(labelSV.shape[0])
    m,n=dataMat.shape
    errorCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict=kernelEval.T*np.multiply(labelSV,alphas[svInd])+b
        if(np.sign(predict)!=np.sign(labelArr[i])):
            errorCount+=1
    print 'the trainning error rate is :%f'%(1.0*errorCount/m)
    dataArr,labelArr=loadDataSet('testSetRBF2.txt')
    m,n=dataMat.shape
    dataMat=np.mat(dataArr)
    labelMat=np.mat(labelArr).transpose()
    errorCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,dataMat[i,:],('rbf',k1))
        predict=kernelEval.T*np.multiply(labelSV,alphas[svInd])+b
        if(np.sign(predict)!=np.sign(labelArr[i])):
            errorCount+=1
    print 'the test error rate is :%f'%(1.0*errorCount/m)
    

In [13]:
testRbf(k1=1)

L=H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 1
fullSet, iter: 0 i:3, pairs changed 2
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 5
L=H
fullSet, iter: 0 i:8, pairs changed 5
fullSet, iter: 0 i:9, pairs changed 5
fullSet, iter: 0 i:10, pairs changed 6
fullSet, iter: 0 i:11, pairs changed 6
fullSet, iter: 0 i:12, pairs changed 6
fullSet, iter: 0 i:13, pairs changed 7
L=H
fullSet, iter: 0 i:14, pairs changed 7
fullSet, iter: 0 i:15, pairs changed 8
fullSet, iter: 0 i:16, pairs changed 8
fullSet, iter: 0 i:17, pairs changed 9
j not moving enough
fullSet, iter: 0 i:18, pairs changed 9
fullSet, iter: 0 i:19, pairs changed 10
fullSet, iter: 0 i:20, pairs changed 10
fullSet, iter: 0 i:21, pairs changed 11
fullSet, iter: 0 i:22, pairs changed 11
j not moving enough
fullSet, iter: 0 i:23, pairs changed 11
fullSet, iter: 0 

# 6.6 示例：手写识别问题回顾

In [14]:
def img2vector(filename):
    returnVect = np.zeros((1,1024))
    fr = open(filename)
    for i in range(32):
        lineStr = fr.readline()
        for j in range(32):
            returnVect[0,32*i+j] = int(lineStr[j])
    return returnVect

def loadImage(dirName):
    from os import listdir
    hwLabel=[]
    trainingFileList=listdir(dirName)
    del(trainingFileList[0])
    m=len(trainingFileList)
    trainingMat=np.zeros((m,1024))
    for i in range(m):
        fileNameStr=trainingFileList[i]
        fileStr=fileNameStr.split('.')[0]
        classNumstr=int(fileStr.split('_')[0])
        if(classNumstr==9):
            hwLabel.append(-1)
        else:
            hwLabel.append(1)
        trainingMat[i-1,:]=img2vector('%s/%s'%(dirName,fileNameStr))
    return trainingMat,hwLabel

def testDigits(kTup=('rbf',10)):
    dataArr,labelArr=loadImage('trainingDigits')
    b,alphas=smoP(dataArr,labelArr,200,0.0001,10000,kTup)
    dataMat=np.mat(dataArr)
    labelMat=np.mat(labelArr).transpose()
    svInd=np.nonzero(alphas.A>0)[0]
    sVs=dataMat[svInd]
    labelSV=labelMat[svInd]
    print 'there are %d Support Vectors'%(labelSV.shape[0])
    m,n=dataMat.shape
    errorCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,dataMat[i,:],kTup)
        predict=kernelEval.T*np.multiply(labelSV,alphas[svInd])+b
        if(np.sign(predict)!=np.sign(labelArr[i])):
            errorCount+=1
    print 'the trainning error rate is :%f'%(1.0*errorCount/m)
    dataArr,labelArr=loadImage('testDigits')
    dataMat=np.mat(dataArr)
    labelMat=np.mat(labelArr).transpose()
    m,n=dataMat.shape
    errorCount=0
    for i in range(m):
        kernelEval=kernelTrans(sVs,dataMat[i,:],kTup)
        predict=kernelEval.T*np.multiply(labelSV,alphas[svInd])+b
        if(np.sign(predict)!=np.sign(labelArr[i])):
            errorCount+=1
    print 'the test error rate is :%f'%(1.0*errorCount/m)
    

In [15]:
testDigits(kTup=('lin', 0))

L=H
fullSet, iter: 0 i:0, pairs changed 0
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 4
fullSet, iter: 0 i:5, pairs changed 5
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 7
fullSet, iter: 0 i:10, pairs changed 8
fullSet, iter: 0 i:11, pairs changed 9
fullSet, iter: 0 i:12, pairs changed 10
fullSet, iter: 0 i:13, pairs changed 10
fullSet, iter: 0 i:14, pairs changed 11
fullSet, iter: 0 i:15, pairs changed 12
L=H
fullSet, iter: 0 i:16, pairs changed 12
L=H
fullSet, iter: 0 i:17, pairs changed 12
fullSet, iter: 0 i:18, pairs changed 12
L=H
fullSet, iter: 0 i:19, pairs changed 12
L=H
fullSet, iter: 0 i:20, pairs changed 12
L=H
fullSet, iter: 0 i:21, pairs changed 12
L=H
fullSet, iter: 0 i:22, pairs changed 12
L=H
fullSet, iter: 0 i:23, pairs changed 12
fullSet, iter: 0 i:24, pairs c